In [ ]:
import os
import numpy as np
import tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet import preprocess_input

In [ ]:
base_model = tensorflow.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

dataset_dir = "D:\\face_age"
images = []
labels = []
for age_folder in os.listdir(dataset_dir):
    if os.path.isdir(os.path.join(dataset_dir, age_folder)):
        age = int(age_folder)
        for image_file in os.listdir(os.path.join(dataset_dir, age_folder)):
            img_path = os.path.join(dataset_dir, age_folder, image_file)
            try:
                img = load_img(img_path, target_size=(224, 224))
                img_array = img_to_array(img)
                img_array = preprocess_input(img_array) 
                images.append(img_array)
                labels.append(age)
            except Exception as e:
                print(f"Error processing image '{img_path}': {e}")

images = np.array(images)
labels = np.array(labels)

In [ ]:
class MAEStopCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MAEStopCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        if logs.get('mae') <= self.threshold:
            print(f"\nStopping training as MAE reached {self.threshold}")
            self.model.stop_training = True

In [ ]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation=None)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
callback = MAEStopCallback(threshold=1.0)
model.fit(images, labels, epochs=50, batch_size=32, callbacks = [callback])

In [ ]:
new_img_path = "D:\\face_age\\075\\2108.png"
new_img = load_img(new_img_path, target_size=(224, 224))
new_img_array = img_to_array(new_img)
new_img_array = np.expand_dims(new_img_array, axis=0)
new_img_array = preprocess_input(new_img_array)  

prediction = model.predict(new_img_array)
predicted_age = int(np.round(prediction[0][0]))

print("Predicted Age:", predicted_age)

In [ ]:
def predict_age(face_img):
    face_img = cv2.resize(face_img, (224, 224))
    face_img = img_to_array(face_img)
    face_img = np.expand_dims(face_img, axis=0)
    face_img = preprocess_input(face_img)
    prediction = model.predict(face_img)
    predicted_age = int(np.round(prediction[0][0])) 

    return predicted_age

In [ ]:
import cv2
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        predicted_age = predict_age(face_img)
        if predicted_age >= 18:
            message = f"Age: {predicted_age}. Access Given"
        else:
            message = f"Age: {predicted_age}. Access Denied"
        cv2.putText(frame, message, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.imshow('Face Age Estimation', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()